# Spherical to cartesian

In [20]:
import numpy as np
import scipy
from scipy.special import factorial, factorial2
from math import comb
from anisoap.utils import monomial_iterator

#We are implementing iterations of the form R_{l} = prefact_minus1* z * R_{l-1} + prefact_minus2* r^2 * R_{l-2}
def prefact_minus1(l):
    """
    Parameters:
    - l: int 
    
    Returns:
    - A list of size (2*l -1) corresponding to the prefactor that multiplies the   
    
    For m in [-l, -l+2, ..., l], compute the factor as : 
    sqrt(factorial(l+1-m)/ factorial(l+1+m)) sqrt(factorial(l+m)/ factorial(l-m)) (2*l+1)/(l+1-m) 
    
    """
    m=np.arange(-l,l+1)
    return np.sqrt(factorial(l+1-m)/factorial(l+1+m)) * np.sqrt(factorial(l+m)/factorial(l-m)) * (2*l+1)/(l+1-m)
    
def prefact_minus2(l):
    """
    For m in [-l+1, -l+2, ..., l-1], compute the factor as : 
    sqrt(factorial(l+1-m)/ factorial(l+1+m)) sqrt(factorial(l-1+m)/ factorial(l-1-m)) (l+m)/(l-m+1) 
    """
    m=np.arange(-l+1,l)
    return -1* np.sqrt(factorial(l+1-m)/factorial(l+1+m)) * np.sqrt(factorial(l-1+m)/factorial(l-1-m)) * (l+m)/(l+1-m)

def binom(n, k):
    """
    returns binomial coefficient nCk = math.factorial(n) / (math.factorial(k) * math.factorial(n - k))
    We use math.comb utility to compute this 
    """
    return comb(n, k)


In [83]:
def spherical_to_cartesian2(lmax, num_ns):
    """
    Parameters:
    - lmax: int 
            
    - num_ns: list of ints
        
    Returns:
    - A list with as many entries as in num_ns (corresponding to l=0,1,... lmax+1). Each corresponding entry is an
      array of size (2*l+1, num_ns[l], maxdeg, maxdeg, maxdeg) 
      where maxdeg = l + 2* max(num_ns[l]) and the last three axes correspond to n0, n1, n2 respectively such that
      the entries of the array form the coefficient of the x^n0 y^n1 z^n2 monomial in the expansion of R_l^m
    """

    assert len(num_ns) == lmax + 1

    # Initialize array in which to store all
    # coefficients for each l
    # Usage T_l[m,n,n0,n1,n2]
    T = []
    for l, num_n in enumerate(num_ns):
        maxdeg = l + 2*(num_n-1)
        T_l = np.zeros((2*l+1,num_n,maxdeg+1,maxdeg+1,maxdeg+1))
        T.append(T_l)

    # Initialize array in which to store all coefficients for each l
    # Usage T_l[m,n,n0,n1,n2]
    T[0][0,0,0,0,0] = 1
    for l in range(1,lmax+1):
        prefact = np.sqrt(2) * factorial2(2*l-1) / np.sqrt(factorial(2*l))
        for k in range(l//2+1):
            n1 = 2*k
            n0 = l-n1
            T[l][2*l,0, n0, n1,0] = binom(l, n1) *(-1)**k  
        for k in range((l-1)//2+1):
            n1 = 2*k+1
            n0 = l-n1
            T[l][0,0,n0,n1,0] = binom(l, n1) *(-1)**k
        T[l]*= prefact

    # Run iteration over (l,m) to generate all coefficients for n=0.
    for l in range(1, lmax+1):
        deg = l
        myiter = iter(monomial_iterator.TrivariateMonomialIndices(deg))
        for idx,n0,n1,n2 in myiter:
            print(l)
            a = prefact_minus1(l-1) # length 2l+1 due to m dependence
            b = prefact_minus2(l-1) # length 2l+1 due to m dependence
    
            #(-l+1)+2: (l+1) -2 gets contributions from T[l-1]
            if n0-2>=0:
                print("n0",l)
                T[l][2:2*l-1,0,n0,n1,n2] += b * T[l-2][:,0,n0-2,n1,n2]
            if n1-2>=0:
                print("n1",l)
                T[l][2:2*l-1,0,n0,n1,n2] += b * T[l-2][:,0,n0,n1-2,n2]
            if n2-2>=0:
                print("n2",l)
                T[l][2:2*l-1,0,n0,n1,n2] += b * T[l-2][:,0,n0,n1,n2-2]
            #(-l+1)+1: (l+1) -1 gets contributions from T[l]
            if n2-1>=0:
                print("n2>1",l)
                print(T[l][1:2*l,0,n0,n1,n2].shape, a.shape,T[l-1][:,0,n0,n1,n2-1].shape )
                T[l][1:2*l,0,n0,n1,n2] += a * T[l-1][:,0,n0,n1,n2-1]
#                 a's length needs to be fixed 
                
    # Run the iteration over n
#     B_{n+1,lm} = r^2 B_{nlm} = (x^2+y^2+z^2)B_{nlm}
#     T^{n+1,lm}_{n_0+2,n_1,n_2} += T^{n,lm}_{n_0,n_1n_2}
#     T^{n+1,lm}_{n_0,n_1+2,n_2} += T^{n,lm}_{n_0,n_1n_2}
#     T^{n+1,lm}_{n_0,n_1,n_2+2} += T^{n,lm}_{n_0,n_1n_2}
    for l in range(lmax+1):
        for n in range(1,num_ns[l]):
            deg = l + 2*n # degree of polynomial
            myiter = iter(monomial_iterator.TrivariateMonomialIndices(deg))
            for idx,n0,n1,n2 in myiter:
                # Use recurrence relation to update
                # Warning, if n0-2, n1-2 or n2-2 are negative
                # it might be necessary to add if statements
                # to avoid.
                if n0>=2:
                    T[l][:,n,n0,n1,n2] += T[l][:,n-1,n0-2,n1,n2]
                if n1>=2:
                    T[l][:,n,n0,n1,n2] += T[l][:,n-1,n0,n1-2,n2]
                if n2>=2:
                    T[l][:,n,n0,n1,n2] += T[l][:,n-1,n0,n1,n2-2]

    return T

In [86]:
lmax=2
num_ns=[1,1,2]
test = spherical_to_cartesian2(lmax, num_ns)

1
1
1
n2>1 1
(1,) (1,) (1,)
2
n0 2
2
2
n2>1 2
(3,) (3,) (3,)
2
n1 2
2
n2>1 2
(3,) (3,) (3,)
2
n2 2
n2>1 2
(3,) (3,) (3,)


In [87]:
from anisoap.utils.spherical_to_cartesian import spherical_to_cartesian
ref = spherical_to_cartesian(lmax, num_ns)


In [88]:
for i in range(len(ref)):
    print(np.linalg.norm(ref[i]-test[i]))

0.0
0.0
0.0


# projection_coefficients

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from anisoap.utils.spherical_to_cartesian import spherical_to_cartesian
import numpy as np
try:
    from tqdm import tqdm
except ImportError:
    tqdm = (lambda i, **kwargs: i)

from anisoap.utils import compute_moments_diagonal_inefficient_implementation
from anisoap.utils import quaternion_to_rotation_matrix
from anisoap.representations.radial_basis import RadialBasis
from anisoap.utils import compute_moments_inefficient_implementation

from metatensor import TensorBlock, TensorMap,Labels

In [4]:
from ase.io import read
from rascaline import NeighborList
import ase
frames = read('/Users/jigyasa/scratch/data_papers/data/water/dataset/water_randomized_1000.xyz', ':2')
frames2 = read("/Users/jigyasa/scratch/metatensor-examples/data/hamiltonian/ethanol-hamiltonian/ethanol_4500.xyz",":1")
frames3= [ase.build.molecule('NH3')]
frames = frames+frames2+frames3
for f in frames: 
    f.pbc=True
    f.cell=[4,4,4]

In [5]:
from anisoap.utils.moment_generator import *
from anisoap.utils.quaternion_to_rotation_matrix import  quaternion_to_rotation_matrix
from anisoap.representations.radial_basis import RadialBasis
from anisoap.utils import compute_moments_inefficient_implementation
from itertools import product
import anisoap.representations.radial_basis as radial_basis

In [6]:
#hypers
rcut = 4
lmax = 4
radial_basis_name = 'monomial'
radial_hypers={}
radial_hypers['radial_basis'] = radial_basis_name.lower() # lower case
radial_hypers['radial_gaussian_width'] = 0.2
radial_hypers['max_angular'] = lmax

radial_basis = RadialBasis(**radial_hypers)

num_ns = []
for l in range(lmax+1):
#     num_ns.append(lmax + 1 - l)
    num_ns.append(1)
sph_to_cart = spherical_to_cartesian(lmax, num_ns)

In [47]:
# from anisoap.representations.projection_coefficients_modified import *
# dp = DensityProjectionCalculator(lmax,
#                  'monomial',
#                  rcut,
#                  compute_gradients=False,
#                  subtract_center_contribution=False,
#                  radial_gaussian_width = None)
# dp.transform(frames)

ModuleNotFoundError: No module named 'anisoap.representations.projection_coefficients_modified'

In [7]:
num_frames = len(frames)
num_atoms_per_frame = np.zeros((num_frames),int)
species = set()

for i,f in enumerate(frames):
    num_atoms_per_frame[i] = len(f)
    for atom in f:
        species.add(atom.number)

num_atoms = sum(num_atoms_per_frame)
species = sorted(species)

In [8]:
frame_to_global_atom_idx = np.zeros((num_frames),int)
for n in range(1,num_frames):
    frame_to_global_atom_idx[n] = num_atoms_per_frame[n-1] + frame_to_global_atom_idx[n-1]


In [58]:
quaternions = np.zeros((num_atoms,4))
ellipsoid_lengths = np.zeros((num_atoms, 3))

for i in range(num_atoms):
    quaternions[i]= [0, 0, np.sin(np.pi/4), np.cos(np.pi/4)]
    ellipsoid_lengths[i] = [0.5,0.3,0.4]

# Convert quaternions to rotation matrices
rotation_matrices = np.zeros((num_atoms,3,3))
for i, quat in enumerate(quaternions):
    rotation_matrices[i] = quaternion_to_rotation_matrix(quat)


In [9]:
nl = NeighborList(rcut, True).compute(frames)
keys=np.array(nl.keys.asarray(), dtype=int)
keys=[tuple(i)+(l,) for i in keys for l in range(lmax+1)]

In [10]:
nl.block(0).values.shape

(150, 3, 1)

In [11]:
def pairwise_aniso_expansion(neighbor_list, species, frame_to_global_atom_idx, rotation_matrices, ellipsoid_lengths, radial_basis):
    """
    Function to compute the pairwise expansion <anlm|rho_ij> by combining the moments and the spherical to Cartesian 
    transformation
    --------------------------------------------------------
    Parameters:
    
    neighbor_list : Equistore TensorMap 
        Full neighborlist with keys (species_1, species_2) enumerating the possible species pairs.
        Each block contains as samples, the atom indices of (first_atom, second_atom) that correspond to the key,
        and block.value is a 3D array of the form (num_samples, num_components,properties), with num_components=3 
        corresponding to the (x,y,z) components of the vector from first_atom to second_atom.
        Depending on the cutoff some species pairs may not appear. Self pairs are not present but in PBC,
        pairs between copies of the same atom are accounted for.
       
    species: list of ints
        List of atomic numbers present across the data frames 
        
    frame_to_global_atom_idx: list of ints
        The length of the list equals the number of frames, each entry enumerating the number of atoms in
        corresponding frame
        
    rotation_matrices: np.array of dimension ((num_atoms,3,3))
        
    ellipsoid_lengths: np.array of dimension ((num_atoms,3))
    
    radial_basis : Instance of the RadialBasis Class
        anisoap.representations.radial_basis.RadialBasis that has been instantiated appropriately with the 
        cutoff radius, radial basis type.
    -----------------------------------------------------------
    Returns: 
        An Equistore TensorMap with keys (species_1, species_2, l) where ("species_1", "species_2") is key in the 
        neighbor_list and "l" is the angular channel.
        Each block of this tensormap has the same samples as the corresponding block of the neighbor_list.
        block.value is a 3D array of the form (num_samples, num_components, properties) where num_components 
        form the 2*l+1 values for the corresponding angular channel and the properties dimension corresponds to 
        the radial channel.
        
    """ 
    tensorblock_list = []
    tensormap_keys = []

    for center_species in species:
        for neighbor_species in species:
            if (center_species, neighbor_species) in neighbor_list.keys:
                values_ldict = {l:[] for l in range(lmax+1)}
                nl_block = neighbor_list.block(species_first_atom=center_species, species_second_atom=neighbor_species)

                for isample, nl_sample in enumerate(nl_block.samples):
                    frame_idx, i,j = nl_sample["structure"], nl_sample["first_atom"], nl_sample["second_atom"]
                    i_global = frame_to_global_atom_idx[frame_idx] +i
                    j_global = frame_to_global_atom_idx[frame_idx] +j

                    r_ij = np.asarray([nl_block.values[isample,0], nl_block.values[isample,1],nl_block.values[isample,2]]).reshape(3,)
    #                 r_ij = pos_i - positions[j_global]

                    rot = rotation_matrices[j_global]
                    lengths = ellipsoid_lengths[j_global]
                    precision, center =  radial_basis.compute_gaussian_parameters(r_ij, lengths, rot)
    #                 moments = compute_moments_inefficient_implementation(precision, center, maxdeg=lmax)

                    for l in range(lmax+1):
                        moments = np.ones(sph_to_cart[l].shape[-3:])
                        values_ldict[l].append(np.einsum("mnpqr, pqr->mn", sph_to_cart[l], moments))


                for l in range(lmax+1):
                    block = TensorBlock(values = np.asarray(values_ldict[l]), 
                                        samples = nl_block.samples, #as many rows as samples
                                        components = [Labels(['spherical_component_m'], np.asarray([list(range(-l,l+1))], np.int32).reshape(-1,1) )],
                                        properties = Labels(['n'],np.asarray(list(range(num_ns[l])), np.int32).reshape(-1,1))
                                       )
                    tensorblock_list.append(block)


    pairwise_aniso_feat = TensorMap(Labels(["species_center", "species_neighbor", "angular_channel"],np.asarray(keys, dtype=np.int32)), tensorblock_list)
    return pairwise_aniso_feat

In [12]:
pairwise_aniso_feat = pairwise_aniso_expansion(nl, species, frame_to_global_atom_idx, rotation_matrices, ellipsoid_lengths, radial_basis)

In [13]:
pairwise_aniso_feat

TensorMap with 50 blocks
keys: ['species_center' 'species_neighbor' 'angular_channel']
              1                1                 0
              1                1                 1
              1                1                 2
           ...
              8                6                 2
              8                6                 3
              8                6                 4

In [380]:
moments.shape

(5, 5, 5)

In [367]:
sph_to_cart[l].shape

(9, 1, 5, 5, 5)

In [51]:
def contract_pairwise_feat(pair_aniso_feat, species):
    """
    Function to sum over the pairwise expansion \sum_{j in a} <anlm|rho_ij> = <anlm|rho_i>
    --------------------------------------------------------
    Parameters:
    
    pair_aniso_feat : Equistore TensorMap 
        TensorMap returned from "pairwise_aniso_expansion()" with keys (species_1, species_2,l) enumerating 
        the possible species pairs and the angular channels.
    
    species: list of ints
        List of atomic numbers present across the data frames 
        
    -----------------------------------------------------------
    Returns: 
        An Equistore TensorMap with keys (species, l) "species" takes the value of the atomic numbers present 
        in the dataset and "l" is the angular channel.
        Each block of this tensormap has as samples ("structure", "center") yielding the indices of the frames 
        and atoms that correspond to "species" are present.
        block.value is a 3D array of the form (num_samples, num_components, properties) where num_components 
        take on the same values as in the pair_aniso_feat_feat.block .  block.properties now has an additional index 
        for neighbor_species that corresponds to "a" in <anlm|rho_i>
        
    """ 
    aniso_keys = list(set([tuple(list(x)[:1]+list(x)[2:]) for x in keys]))
    # Select the unique combinations of pair_aniso_feat.keys["species_center"] and 
    # pair_aniso_feat.keys["angular_channel"] to form the keys of the single particle centered feature 
    aniso_keys.sort()
    aniso_blocks = []
    property_names = pair_aniso_feat.property_names + ('neighbor_species',)
    
    for key in aniso_keys:
        contract_blocks=[]
        contract_properties=[]
        contract_samples=[]
        # these collect the values, properties and samples of the blocks when contracted over neighbor_species.
        # All these lists have as many entries as len(species). 
        
        for ele in species:
            blockidx = pair_aniso_feat.blocks_matching(species_neighbor= ele)
            # indices of the blocks in pair_aniso_feat with neighbor species = ele
            sel_blocks = [pair_aniso_feat.block(i) for i in blockidx if key==tuple(list(pair_aniso_feat.keys[i])[:1]+list(pair_aniso_feat.keys[i])[2:])]
            if not len(sel_blocks):
#                 print(key, ele, "skipped") # this block is not found in the pairwise feat
                continue
            assert len(sel_blocks)==1 
            
            # sel_blocks is the corresponding block in the pairwise feat with the same (species_center, l) and 
            # species_neighbor = ele thus there can be only one block corresponding to the triplet (species_center, species_neighbor, l)
            block = sel_blocks[0]
            
            pair_block_sample = list(zip(block.samples['structure'], block.samples['first_atom']))
            # Takes the structure and first atom index from the current pair_block sample. There might be repeated
            # entries here because for example (0,0,1) (0,0,2) might be samples of the pair block (the index of the
            # neighbor atom is changing but for both of these we are keeping (0,0) corresponding to the structure and 
            #first atom. 

            struct, center = np.unique(block.samples['structure']), np.unique(block.samples['first_atom'])
            possible_block_samples = list(product(struct,center))
            # possible block samples contains all *unique* possible pairwise products between structure and atom index
            # From here we choose the entries that are actually present in the block to form the final sample
            
            block_samples=[]
            block_values = []

            for isample, sample in enumerate(possible_block_samples):
                sample_idx = [idx for idx, tup in enumerate(pair_block_sample) if tup[0] ==sample[0] and tup[1] == sample[1]]
                # all samples of the pair block that match the current sample 
                # in the example above, for sample = (0,0) we would identify sample_idx = [(0,0,1), (0,0,2)] 
                if len(sample_idx)==0:
                    continue
    #             #print(key, ele, sample, block.samples[sample_idx])
                block_samples.append(sample)
                block_values.append(block.values[sample_idx].sum(axis=0)) #sum over "j"  for given ele
                
                # block_values has as many entries as samples satisfying (key, neighbor_species=ele). 
                # When we iterate over neighbor species, not all (structure, center) would be present
                # Example: (0,0,1) might be present in a block with neighbor_species = 1 but no other pair block 
                # ever has (0,0,x) present as a sample- so (0,0) doesnt show up in a block_sample for all ele
                # so in general we have a ragged list of contract_blocks
            
            contract_blocks.append(block_values)
            contract_samples.append(block_samples)
            contract_properties.append([tuple(p)+(ele,) for p in block.properties])
            #this adds the "ele" (i.e. neighbor_species) to the properties dimension
        
#         print(len(contract_samples))
        all_block_samples= sorted(list(set().union(*contract_samples))) 
        # Selects the set of samples from all the block_samples we collected by iterating over the neighbor_species
        # These form the final samples of the block! 

        all_block_values = np.zeros(((len(all_block_samples),)+ block.values.shape[1:]+(len(contract_blocks),)))
        # Create storage for the final values - we need as many rows as all_block_samples, 
        # block.values.shape[1:] accounts for "components" and "properties" that are already part of the pair blocks
        # and we dont alter these
        # len(contract_blocks) - adds the additional dimension for the neighbor_species since we accumulated 
        # values for each of them as \sum_{j in ele} <|rho_ij>
        #  Thus - all_block_values.shape = (num_final_samples, components_pair, properties_pair, num_species)
        
        for iele, elem_cont_samples in enumerate(contract_samples):
            # This effectively loops over the species of the neighbors
            # Now we just need to add the contributions to the final samples and values from this species to the right
            # samples
            nzidx=[i for i in range(len(all_block_samples)) if all_block_samples[i] in elem_cont_samples]
            # identifies where the samples that this species contributes to, are present in the final samples
#             print(apecies[ib],key, bb, all_block_samples)
            all_block_values[nzidx,:,:,iele] = contract_blocks[iele]

        new_block = TensorBlock(values = all_block_values.reshape(all_block_values.shape[0],all_block_values.shape[1] ,-1),
                                        samples = Labels(['structure', 'center'], np.asarray(all_block_samples, np.int32)), 
                                         components = block.components,
                                         properties= Labels(list(property_names), np.asarray(np.vstack(contract_properties),np.int32))
                                         )

        aniso_blocks.append(new_block)
    aniso = TensorMap(Labels(['species_center','angular_channel'],np.asarray(aniso_keys,dtype=np.int32)), aniso_blocks)

    return aniso


In [52]:
contract_pairwise_feat(pairwise_aniso_feat, species)

TensorMap with 20 blocks
keys: ['species_center' 'angular_channel']
              1                0
              1                1
              1                2
           ...
              8                2
              8                3
              8                4